# Intro to Full Stack: Databases

This applied practice will be focusing mainly on interacting with SQL databases
through the widely used Object-Relational Mapping (ORM) library, SQLAlchemy.

## Miscellaneous

Run through these after starting/restarting the notebook kernel.

### Auto-Reload Imports

In [1]:
# Auto-reload imports so you don't have to keep restarting the notebook kernel.
%load_ext autoreload
%autoreload complete --print

### Enable Printing SQL Statements Made by SQLAlchemy

In [2]:
import logging

# Log SQL statements made by SQLAlchemy, I suggest you turn this off once you get a hang of it.
LOG_SQLALCHEMY = True

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(
    logging.INFO if LOG_SQLALCHEMY else logging.WARNING
)

### Reset Database File

In [ ]:
import shutil
from pathlib import Path

cwd = Path.cwd()
# Overwrite the workshop database with the prepared example database.
shutil.copy(cwd / "src" / "example.db", cwd / "my_db.db")

### FYI: Create All Tables

This won't log any SQL `CREATE TABLE` since the prepared example database already
contains the tables. Nonetheless, when you write your own code, you have to do
this at least once to create the tables for a new database.

In [ ]:
from src import Session, Base

# Create all tables.
with Session.begin() as sess:
    Base.metadata.create_all(sess.bind)


## Example: CRUD with Users

In [ ]:
from src.utils import random_name, hash_passwd

EXAMPLE_USERNAME = random_name()
EXAMPLE_PASSWD = hash_passwd("password")
EXAMPLE_USER_ID = None  # Placeholder for the user ID which will be assigned later.
print(f"[Username for Example]\n{EXAMPLE_USERNAME}\n")
print(f"[Password for Example]\n{EXAMPLE_PASSWD}")

### Examining the User Schema

First, view the `User` database schema in [`src/models/user.py`](src/models/user.py).

### Creating a User

Notice something interesting?

In [ ]:
from src import Session, User

with Session() as sess:
    user = User(username=EXAMPLE_USERNAME, password=EXAMPLE_PASSWD)
    sess.add(user)

    print(f"[`user` Before Commit]\n{user}\n\n", flush=True)
    print("[Committing...]", flush=True)
    sess.commit()
    print("[Committed]", flush=True)

    print("\n\n[Reading from DB...]", flush=True)
    value = str(user)
    print(f"\n\n[`user` After Commit]\n{value}", flush=True)

    EXAMPLE_USER_ID = user.id

Until `sess.commit()` is called, the instance of `User` we created isn't populated
with default values yet. Rather, values like `id` change from None to their actual values only after the commit, even if the field is non-nullable. This means we can't rely on the instance until after the commit.

## Reading a User

### By ID

### By Other `index=True` Columns

### Searching for Users

### Filtering Users

## Updating a User

### After Being Found Above

### Updating Multiple Users

## Deleting a User

## Your Turn: Transactions CRUD

### Writing the Schema

Fill in the `Transaction` model in [`src/models/transaction.py`](src/models/transaction.py).

### Testing Create

### Testing Read

#### Find by ID

#### Filter by Amount

### Testing Update

### Testing Delete
